In [6]:
from utils import get_lexicon, chars_to_remove
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
lex = get_lexicon()

f = "sample.txt"
with open(f, "r") as file:
    sample = file.read()

def preprocess(s:str):
    for c in chars_to_remove:
        s = s.replace(c, "")
    s = s.casefold()
    return s

def count_variants(s: str, lex):
    counts = dict()
    for word in preprocess(s).split():
        variant = lex.get(word, None)
        if not variant:
            continue
        logging.info(f"Found word {word}, presumed variant: {variant}.")
        counts[variant] = counts.get(variant, 0) + 1
    return counts

2022-02-16 09:06:14,462 Weird formatting with >3 slashes:
	_: kabbalah / _v: cabala / _v: kabbala / _V: kabala / _V: cabbala / _V: cabbalah


In [7]:
count_variants(sample, lex)


2022-02-16 09:06:15,029 Found word airplanes, presumed variant: A.
2022-02-16 09:06:15,032 Found word traveler, presumed variant: A.
2022-02-16 09:06:15,032 Found word traveler, presumed variant: A.
2022-02-16 09:06:15,033 Found word traveler, presumed variant: A.
2022-02-16 09:06:15,033 Found word traveler, presumed variant: A.
2022-02-16 09:06:15,034 Found word traveler, presumed variant: A.
2022-02-16 09:06:15,034 Found word analyses, presumed variant: B.
2022-02-16 09:06:15,035 Found word et, presumed variant: A.


{'A': 7, 'B': 1}

In [8]:
count_variants("We analysed Her Majesties' biscuits and found out they crumble ever so scrumptiously", lex)

2022-02-16 09:06:15,392 Found word analysed, presumed variant: B.


{'B': 1}